In [1]:
import os
import pydicom
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Configs
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15
DATA_DIR = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images/'

# Load CSVs
train_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train.csv')
series_desc_df = pd.read_csv('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv')

# Labels
label_cols = train_df.columns[1:]
label_map = {'Normal/Mild': 0, 'Moderate': 1, 'Severe': 2}

def encode_labels(row):
    return [label_map.get(row[col], 0) for col in label_cols]
train_df['encoded_labels'] = train_df.apply(encode_labels, axis=1)

def get_max_severity(encoded_labels):
    return max(encoded_labels)
train_df['max_severity'] = train_df['encoded_labels'].apply(get_max_severity)


# Series IDs per study
def get_series_ids(study_id):
    sub_df = series_desc_df[series_desc_df['study_id'] == study_id]
    views = {'Sagittal T1': None, 'Sagittal T2/STIR': None, 'Axial T2': None}
    for view in views:
        found = sub_df[sub_df['series_description'].str.contains(view, case=False)]
        if not found.empty:
            views[view] = found.iloc[0]['series_id']
    return views

# Load DICOM without augmentation
def load_dicom_image(path):
    dcm = pydicom.dcmread(path)
    img = dcm.pixel_array.astype(np.float32)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = img / np.max(img)
    return img

# Load images for a study and average them over all available images per view
def load_study_images(study_id):
    views = get_series_ids(study_id)
    images = []
    for view in ['Sagittal T1', 'Sagittal T2/STIR', 'Axial T2']:
        series_id = views[view]
        if pd.isna(series_id):
            images.append(np.zeros((IMG_SIZE, IMG_SIZE, 3)))  # No image, so append blank image
        else:
            series_path = os.path.join(DATA_DIR, str(study_id), str(series_id))
            instances = sorted(os.listdir(series_path))
            view_images = []
            for instance in instances:
                img_path = os.path.join(series_path, instance)
                view_images.append(load_dicom_image(img_path))  # Load all images for this view
            if view_images:
                # Average pooling over all images in this view
                avg_image = np.mean(view_images, axis=0)
                images.append(avg_image)
            else:
                images.append(np.zeros((IMG_SIZE, IMG_SIZE, 3)))  # If no images, append blank image
    return images

# Data generator without augmentation
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size=BATCH_SIZE, shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.df))
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        batch_ids = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_ids]
        X1, X2, X3, y = [], [], [], []
        for _, row in batch_df.iterrows():
            imgs = load_study_images(row['study_id'])
            # Append the average pooled image for each view
            X1.append(np.array(imgs[0]))  # Sagittal T1
            X2.append(np.array(imgs[1]))  # Sagittal T2/STIR
            X3.append(np.array(imgs[2]))  # Axial T2
            y.append(row['encoded_labels'])
        return (np.array(X1), np.array(X2), np.array(X3)), to_categorical(np.array(y), num_classes=3)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Backbone creation with fine-tuning last 50 layers
def create_backbone():
    base = applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3), pooling='avg')
    for layer in base.layers[-50:]:
        layer.trainable = True
    return base

# Build Multi-View CNN
def build_mvcnn():
    input1 = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    input2 = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    input3 = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    backbone = create_backbone()
    feat1 = backbone(input1)
    feat2 = backbone(input2)
    feat3 = backbone(input3)
    merged = layers.Concatenate()([feat1, feat2, feat3])
    x = layers.Dense(512, activation='relu')(merged)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(len(label_cols)*3, activation='softmax')(x)
    output = layers.Reshape((len(label_cols), 3))(output)
    model = models.Model(inputs=[input1, input2, input3], outputs=output)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Train/Validation Split
train_ids, val_ids = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['max_severity'])
 

train_gen = DataGenerator(train_ids)
val_gen = DataGenerator(val_ids)

# Build and train model
model = build_mvcnn()

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint('best_model_Adam.keras', monitor='val_loss', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, min_lr=1e-6, verbose=1)
]

# Train
model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Save final model
model.save('mvc_MobileNetV2_no_aug_400k.h5')


2025-06-21 13:43:11.344960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750513391.602766      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750513391.679299      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1750513406.544929      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


I0000 00:00:1750513572.398804      59 service.cc:148] XLA service 0x7f214c002350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750513572.399894      59 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750513581.014150      59 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1750513590.392451      59 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750513590.589877      59 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
I0000 00:00:1750513610.426135      59 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 33s/step - accuracy: 0.6629 - loss: 0.8180 
Epoch 1: val_loss improved from inf to 0.73392, saving model to best_model_Adam.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 2146s 42s/step - accuracy: 0.6641 - loss: 0.8159 - val_accuracy: 0.7694 - val_loss: 0.7339 - learning_rate: 1.0000e-04
Epoch 2/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 23s/step - accuracy: 0.7793 - loss: 0.5561 
Epoch 2: val_loss improved from 0.73392 to 0.71864, saving model to best_model_Adam.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 1330s 27s/step - accuracy: 0.7792 - loss: 0.5563 - val_accuracy: 0.7654 - val_loss: 0.7186 - learning_rate: 1.0000e-04
Epoch 3/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 21s/step - accuracy: 0.7785 - loss: 0.5298 
Epoch 3: val_loss improved from 0.71864 to 0.68468, saving model to best_model_Adam.keras
49/49 ━━━━━━━━━━━━━━━━━━━━ 1316s 27s/step - accuracy: 0.7786 - loss: 0.5296 - val_accuracy: 0.7674 - val_loss: 0.6847 - learning_rate: 1.0000e-04
Epoch 4/15
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - 